[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/accdavlo/calcolo-scientifico/blob/main/codes/FEM_with_FEniCS.ipynb)

# Finite element with FEniCS
In this notebook we will use the FEniCS library to solve the Poisson problem in 2D con complex geometries.

[FEniCS project](https://fenicsproject.org/) is a Finite element library which allows to use a very high level language to define complex problems, helping mathematicians getting quickly done with their ideas.
Lately the developers of FEniCS have moved to the new version FEniCSx, but we will use the old one, where some functions are of simpler use.

So, be careful when you install it on your laptop: choose the right version! FEniCS 2018 has also compatibility with `mshr` a library that allows to generate meshes in python. We will stick to this combo.

In Google Colab, Francesco Ballarin has developed a library [FEM on Colab](https://fem-on-colab.github.io/) that allows to quickly install the packages on Google remote servers. We will use it in the next cell.
Still, this is highly discouraged for local computers. It might work, but you have less control on your installation!



In [1]:
# Installing FEniCS (dolfin) on the Google Colab servers
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin
from dolfin import *

--2025-04-04 17:45:53--  https://fem-on-colab.github.io/releases/fenics-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4180 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.08K  --.-KB/s    in 0s      

2025-04-04 17:45:53 (26.5 MB/s) - ‘/tmp/fenics-install.sh’ saved [4180/4180]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/1f33a7ab/releases/pybi

ModuleNotFoundError: No module named 'dolfin'

In [ ]:
# Setting some plotting styles
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12, 9)

In [ ]:
# Importing some libraries
from dolfin import * # This is the core of FEniCS and it contains all the FEM functions we will need
from ufl_legacy.geometry import * # This helps in designing geometries
from dolfin.cpp.mesh import *     # This handles meshes
from mshr import *                # This generates meshes

In [ ]:
# First step: generate a mesh

# Define a domain
domain = Rectangle(Point(0, 0), Point(1, 1))

# Define a mesh with N points on an edge
mesh = generate_mesh(domain, 32)

# Plot the mesh
plot(mesh)

Now the real Finite element experience.

Say we want to solve Poisson on the square with Dirichlet homogenous boundary conditions on all sides except the top one where we use Neumann homogeneous BC
$$
\int_\Omega \nabla u \cdot \nabla v \mathrm{d}x = \int_\Omega fv \mathrm{d}x
$$
and we set $f\equiv 1$.

In [ ]:
# First of all we define the type of Finite element we want to use, this is the equivalent of the reference element we have used in 1D
# 1 is the degree of the used polynomials, one can also try with something larger
V_element = FiniteElement('Lagrange', triangle, 1)

# Then we define the functional space, which depends on the mesh and on the reference element V_element
V = FunctionSpace(mesh, V_element)

# Now we define two types of functions: trial functions and test functions.
# Trial functions are the one that will be used in place of the unknown u of the linear system that sooner or later we will need to generate. They correspond to the columns of the linear system
# Test functions are the ones against which the problem is tested, they will define the rows of the system.
u = TrialFunction(V)
v = TestFunction(V)

# We define the boundary conditions, we need the functional space, the value and where we want to put them to zero (the three sides of interest)
zero_BC = DirichletBC(V, 0.,
                      "on_boundary && \
                      (x[0]<DOLFIN_EPS | x[0]>1.0-DOLFIN_EPS |\
                      x[1]<DOLFIN_EPS)")

# For Neumann homogeneous bcs we do not need to do anything

# List of all boundaries
bcs = [zero_BC]

# Constant function that can be used by FEniCS to be integrated
f = Constant(1.)


Here the weak problem in its majesty: we define the left-hand-side of the problem as the scalar product between the gradients of the test and trial function

`*dx` means that we are integrating over the whole domain

Similarly for the rhs, we compute the integral of `f*v*dx`

In [ ]:

lhs = inner(grad(u),grad(v))*dx
rhs = f*v*dx

print(type(lhs))

These are not yet matrices, but ufl forms, FEniCS knows how to deal with these objects to automatically assemble the matrices/vectors in a sparse way within a dolfin type that is derived from the `PETSc` library (under the hood it's using some C++ and Fortran libraries)

In [ ]:
LHS = assemble(lhs)
RHS = assemble(rhs)

print(type(LHS))
print(type(RHS))

Time to apply the boundary conditions!
As for the 1D case, we change the Dirichlet DoFs equations by directly modifying the lhs matrix and the rhs vector. This creates fake equations for those dofs that assing the appropriate Dirichlet boundaries

In [ ]:
zero_BC.apply(LHS)
zero_BC.apply(RHS)

Finally, we can solve the problem. By default the linear solver of FEniCS is set to an incomplete LU decomposition, but one can manually change it.


In [ ]:
# Third type of functions after TrialFunction, TestFunction

# Function(V) is a simple FEM function: it stores the coefficients of the basis functions and it reference to all the functional space structures for complicated steps
u_sol = Function(V)

# We solve the linear system LHS*u=RHS -> syntax follows this order
# Careful that here the solution of a linear system is not a Function, but a vector, so we have to assign it to the vector of u_sol
solve(LHS, u_sol.vector(), RHS)

# Finally we plot our solution
pp=plot(u_sol)
plt.colorbar(pp)

### Exercise:
1. Try for an exact solution (you can change the problem) to compute the error
1. See what happens when the mesh changes
1. Change the degree
1. Change to non-homogeneous BC

Hints:
1. To deal with complex expressions for the right hand side or other terms in the weak formulations use the function `Expression`, e.g.
```
f = Expression('exp(-10.0*(pow(x[0]-0.75,2) + pow(x[1] - 0.75,2)))',degree = 2)
```
1. To integrate on boundaries instead on the whole domain, use `* ds` to make FEniCS know you want to integrate on the boundaries (in comparison to `* dx` for the integrals on the whole domain)

## Complex geometries
In the next test, we use as a domain a 2D representation of a long channel with a cylinder in it. This is composed of a rectangle minus a circle

In [ ]:
# Defining the channel
channel  = Rectangle(Point(0.,0.),Point(2.2,0.41))
# Defining the cylinder
cylinder_diam = 0.1
cylinder = Circle(Point(0.2,0.2), cylinder_diam/2.)

# Defining the domain
domain = channel-cylinder

# Denerate the mesh
mesh = generate_mesh(domain, 100)
plot(mesh)

print(mesh.num_cells())
print(mesh.num_vertices())

In [ ]:
V_element = FiniteElement('Lagrange', triangle, 1)
V = FunctionSpace(mesh, V_element)
u = TrialFunction(V)
u_sol = Function(V)
v = TestFunction(V)

f = Constant(0.)



### More complex BC!
For each of the boundaries I define a different Dirichlet BC with constant coefficients:
* 0 on the left
* 1 on the right
* 3 on the bottom
* -2 on the top
* -1 on the cylinder  

In [ ]:
## Boundaries
left = 'near(x[0],0)'
right = 'near(x[0],2.2)'
bottom = 'near(x[1],0)'
top = 'near(x[1],0.41)'
cylinder_surf = 'on_boundary && x[0]>0.1 && x[0]<0.3 && x[1]>0.1 && x[1]<0.3'

bc_left = DirichletBC(V, 0., left)
bc_right = DirichletBC(V, 1., right)
bc_bottom = DirichletBC(V, 3., bottom)
bc_top = DirichletBC(V, -2., top)
bc_cylinder = DirichletBC(V, -1., cylinder_surf)

bcs = [bc_left, bc_right, bc_bottom, bc_top, bc_cylinder]

In [ ]:
# Write the weak form
lhs = inner(grad(u),grad(v))*dx
rhs = inner(f,v)*dx

# Assemble matrix
LHS = assemble(lhs)
RHS = assemble(rhs)

# Apply BC for all BCs in the list
for bc in bcs:
    bc.apply(LHS)
    bc.apply(RHS)

# Solve the liner problem
solve(LHS, u_sol.vector(), RHS)

# Plot the solution and add a colorbar
pp=plot(u_sol)
plt.colorbar(pp)

## Exercises
1. More complex domains
1. Different BCs
1. Different problems! Add reaction, add a fourth derivative (using high order FEMs)

Hint:
1. When dealing with very complex boundaries, you might need to use longer functions. The following example might help.
```
def boundary(x, on_boundary):
    d0 = sqrt((x[0]-0.5)**2 + (x[1]-0.5)**2)
    d1 = sqrt((x[0]-1.0)**2 + (x[1]-1.0)**2)
    return on_boundary and (d0 < 0.3 or d1 < 0.3)
bc = DirichletBC(V, 0 , boundary)
```